Import Libraries

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torch.utils.data import DataLoader

import numpy as np

import os
import sys

Device Setup:
If you have an Nvidia GPU, you need to install CUDA
Otherwise, CPU will be used

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Hardware to be used:", device)

if torch.cuda.is_available():
    print(torch.cuda.get_device_name())

Enter the parent path (e.g., "C:\\Users\\{USERNAME}\\alzheimer-project-ai4all")

Enter path to dataset (*.npz)

Enter path to test dataset (e.g., "assets\Kaggle\alzheimer_mri_preprocessed_dataset\Test_Data")

In [3]:
parent_path = r"ENTER PARENT PATH"

dataset_dir = r"resampled\augmented_adasyn_data.npz"
dataset_path = os.path.join(parent_path, dataset_dir)

test_dir = r"assets\Kaggle\alzheimer_mri_preprocessed_dataset\Test_Data\test_data.npz"
test_path = os.path.join(parent_path, test_dir)

sys.path.append(parent_path)

Defining the hyper-parameters

In [4]:
epoches = 25 # implement early stopping
learning_rate = 0.001
batch_size = 8

Loading the datasets

In [5]:
dataset = np.load(dataset_path)
x_train = dataset["images"]
y_train = dataset["labels"]

dataset = np.load(test_path)
x_test = dataset["images"]
y_test = dataset["labels"]


In [ ]:
print(x_train.shape)
print(y_train.shape)

print(x_test.shape)
print(y_test.shape)

In [7]:
train_loader = DataLoader(x_train, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(x_test, batch_size=batch_size, shuffle=False)

Create our model using PyTorch's Convolutional Neural Network

In [8]:
from alzheimersdetection.AlzheimerModel import AlzheimerCNN
size = len(x_train)
model = AlzheimerCNN(size).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

n_total_steps = len(train_loader)

Training Model

In [ ]:
for epoch in range(epoches):
    model.train()
    running_loss = 0.0
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        # Forward
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward() # Backpropogation

        optimizer.step() # Update the weights!!!

        running_loss+=loss.item()

    epoch_loss = running_loss/len(train_loader)
    print(f"Epoch {epoch + 1}/{epoches} - Training Loss: {epoch_loss:.4f}")
